In [302]:
import pandas as pd 
import numpy as np

In [303]:
data_for_NN=pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [304]:
data_for_NN.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [305]:
#We have a dataset with numerous features i will pre process the dataset and build classifier to try and predict
#if the customer will accept a loan offer based on the given features

About this file

Suggest Edits
Attribute Information:

ID : Customer ID

Age : Customer's age in completed years

Experience : #years of professional experience

Income : Annual income of the customer ($000)

ZIP Code : Home Address ZIP code.

Family : Family size of the customer

CCAvg : Avg. spending on credit cards per month ($000)

Education : Education Level.
1: Undergrad;
2: Graduate;
3: Advanced/Professional

Mortgage : Value of house mortgage if any. ($000)

10.Personal Loan : Did this customer accept the personal loan offered in the last campaign?

11.Securities Account : Does the customer have a securities account with the bank?

12.CD Account : Does the customer have a certificate of deposit (CD) account with the bank?

13.Online : Does the customer use internet banking facilities?

14.Credit card : Does the customer use a credit card issued by

In [307]:
data_for_NN.head()


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [308]:
#We can see our dataset doesnt have any null values
#Also we only have numerical data
print(data_for_NN.isnull().sum())
print(data_for_NN.dtypes)
#drop non useful features
## Drop non-predictive columns
#data.drop(columns=["ID", "ZIP Code"], inplace=True)

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64
ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object


In [309]:
#lets check if we have ballanced data regarding the feature 
#Personal Loan : Did this customer accept the personal loan offered in the last campaign?


In [310]:
#We can see that we have highly inbalanced dataset 
print(data_for_NN["Personal Loan"].value_counts()/len(data)*100)

Personal Loan
0    90.4
1     9.6
Name: count, dtype: float64


In [311]:
#We will train firstly a neural network to predict if the customer accepted the banks loan offer or not
#to deal with inbalanced data we will use undersampling method

In [312]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline


# Drop non-predictive columns
data_for_NN.drop(columns=["ID", "ZIP Code"], inplace=True)

# Define features and target
X = data_for_NN.drop(columns=["Personal Loan"])
y = data_for_NN["Personal Loan"]

# Identify numerical columns (Education is already numeric here)
num_features = ["Age", "Experience", "Income", "CCAvg", "Mortgage", "Education"]
# No categorical features to one-hot encode

# Create preprocessing pipeline (scaling numeric features)
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_features)
])

# Create an undersampling strategy (balance the dataset)
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)

# Split data into train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply undersampling to balance the dataset
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Build a simple Neural Network model using MLPClassifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing step
    ('classifier', MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=200, random_state=42))  # Neural network
])

# Train the model
model.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probabilities for ROC-AUC
#prints values of target feature after undersamplying
print("after under samplying the target variable is more balanced")
print(y_train_resampled.value_counts())
# Print classification report
print(classification_report(y_test, y_pred))

# Print ROC-AUC score
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))

after under samplying the target variable is more balanced
Personal Loan
0    768
1    384
Name: count, dtype: int64
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       904
           1       0.71      0.81      0.76        96

    accuracy                           0.95      1000
   macro avg       0.84      0.89      0.86      1000
weighted avg       0.95      0.95      0.95      1000

ROC-AUC Score: 0.975882651179941


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [313]:
#We can see that due to inbalanced datset the model doesnt perform very well for class 1
#even after undersampling.
#Lets try a random forest which deals with inbalanced data better than a neural network 


In [314]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

data_random_forest = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

# Features and target variable
X2 = data_random_forest.drop(columns=['Personal Loan'])  # Features (drop target column)
y2 = data_random_forest['Personal Loan']  # Target variable (Personal Loan)

# Split the data into train and test sets (you can adjust the test_size)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42, stratify=y)

# Scale features if needed (Random Forest doesn't necessarily require scaling, but it's good practice in some cases)
scaler = StandardScaler()
X2_train_scaled = scaler.fit_transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

# Train the Random Forest model with class weight 'balanced' to handle class imbalance
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X2_train_scaled, y2_train)

# Make predictions on the test set
y2_pred = model.predict(X2_test_scaled)

# Evaluate model performance
print("Confusion Matrix:")
print(confusion_matrix(y2_test, y2_pred))

print("\nClassification Report:")
print(classification_report(y2_test, y2_pred))

# check feature importance to understand what the model is learning
print("\nFeature Importances:")
print(model.feature_importances_)

Confusion Matrix:
[[902   2]
 [  7  89]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       904
           1       0.98      0.93      0.95        96

    accuracy                           0.99      1000
   macro avg       0.99      0.96      0.97      1000
weighted avg       0.99      0.99      0.99      1000


Feature Importances:
[0.03106163 0.02462779 0.02310917 0.42501742 0.02785655 0.05592475
 0.21494546 0.10008501 0.03360801 0.00420295 0.04822227 0.00581278
 0.00552619]


In [315]:
#The model is giving exeptional good results on the testing ,which is not always good , overfitting or data leakage 
#can cause this

In [316]:
# Check correlations between features and the target variable to see if there is a very high correlation with any causing leakage
correlation_matrix = data_random_forest.corr()
print(correlation_matrix['Personal Loan'].sort_values(ascending=False))

Personal Loan         1.000000
Income                0.502462
CCAvg                 0.366889
CD Account            0.316355
Mortgage              0.142095
Education             0.136722
Family                0.061367
Securities Account    0.021954
Online                0.006278
CreditCard            0.002802
ZIP Code              0.000107
Experience           -0.007413
Age                  -0.007726
ID                   -0.024801
Name: Personal Loan, dtype: float64


In [317]:
#Seems ok 
#both target classes (0 and 1) in the classification problem give good results so inbalance is not a problem here
#

In [318]:
# Perform cross-validation on the training data
cv_scores = cross_val_score(model, X2_train_scaled, y2_train, cv=5, scoring='accuracy')

# Print cross-validation scores for each fold
print(f"Cross-validation scores: {cv_scores}")

# Print the average cross-validation score
print(f"Average cross-validation score: {cv_scores.mean()}")

Cross-validation scores: [0.98625 0.98375 0.9775  0.9775  0.97875]
Average cross-validation score: 0.9807499999999999


In [319]:
#Good generalization (high cross-validation scores and similar train/test performance)
#Balanced class performance (good results on both class 0 and class 1)
#No overfitting or significant data leakage detected
#Model is just good 